In [ ]:
import matplotlib.pyplot as plt
import librosa
import pickle
import numpy as np
import os
import pickle
import random
import math
import keras
import re
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras import backend as K
import cv2

tf.compat.v1.enable_eager_execution()
config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth=True

sess = tf.compat.v1.Session(config=config)

In [ ]:
def feature_extraction(data_30_sec):
    sampling_rate = 128
    frame_size = 500
    hop_length = 125
    max_after_normalize = 1
    min_after_normalize = -1
    extracted_features = np.zeros(shape=(21, 251, 3))
    
    data_splited = list()
    
    data_splited.append(data_30_sec[0:sampling_rate*10])
    data_splited.append(data_30_sec[sampling_rate*10:sampling_rate*20])
    data_splited.append(data_30_sec[sampling_rate*20:])
    data_splited = np.array(data_splited)
    
    for data_index in range(0, data_splited.shape[0]):
        stft = librosa.stft(data_splited[data_index], n_fft=frame_size, hop_length=hop_length)
        stft_db = librosa.amplitude_to_db(abs(stft))
        norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
        norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize
        
        extracted_features[:, :, data_index] = np.transpose(norm_stft)
    
    
    
    
    
    return extracted_features




def plot_spectrogram(Y, sr, hop_length, y_axis="linear"):
    plt.figure(figsize=(5, 2))
    librosa.display.specshow(Y,
                             sr=sr,
                             hop_length=hop_length,
                             x_axis="time",
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")



In [ ]:
############### Augmentation Data ###############

def trial_file_processor(file_addr, trial):
    
    start_points_list = list()
    start_offset = 3
    end_of_record_starts = 33
    window_length = 10
    for start_point in range(start_offset, end_of_record_starts, 5):
        start_points_list.append([start_point, 
                                start_point+window_length,
                                start_point+2*window_length])
    

    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = 0 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(len(start_points_list), 129, 440, 3))
    
    start_point_number = 0
    for start_points in start_points_list:
        for color_channel in range(0, 3):
            for channel in range(0, 40):
                stft = librosa.stft(subject_file["data"][trial, channel,
                                              128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                              n_fft=256, hop_length=125)
                stft_db = librosa.amplitude_to_db(abs(stft))
                norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
                norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize

                if color_channel_stft is None:
                    color_channel_stft = norm_stft
                else:
                    color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)
                    
            full_norm_stft[start_point_number, : , :, color_channel] = color_channel_stft
            color_channel_stft = None
            
        start_point_number = start_point_number + 1
        
    return full_norm_stft, np.array(subject_file["labels"][trial])


raw_files_addr = "Address/to/preprocessed_DEAP_data"
transformed_files_addr = "Address/to/save/data"
file_names = os.listdir(raw_files_addr)

for file_name in file_names:
    subject_name = file_name.split(".")[0]
    for trial in range(0, 40):
        data, labels = trial_file_processor(raw_files_addr+file_name, trial)
        for sample in range(0, data.shape[0]):
            data_label_dict = {"data":data[sample], "labels":labels}
            
            with open(transformed_files_addr+subject_name+"t"+str(trial)+"s"+str(sample), 'wb') as handle:
                pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
raw_files_addr = "Address/to/preprocessed_DEAP_data"
transformed_files_addr = "A:/Hamavar/DEAP_Dataset/transformed_data/"
file_names = os.listdir(raw_files_addr)

def trial_file_processor(file_addr, trial):
    
    start_points_list = list()
    number_of_windows = 3
    window_length = 10
    for offset in range(0, 30, 2):
        start_points_list.append([offset, offset+window_length, offset+2*window_length])


    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = 0 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(len(start_points_list),129, 440, 3))
    
    start_point_number = 0
    for start_points in start_points_list:
        for color_channel in range(0, 3):
            for channel in range(0, 40):
                stft = librosa.stft(subject_file["data"][trial, channel,
                                              128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                              n_fft=256, hop_length=125)
                stft_db = librosa.amplitude_to_db(abs(stft))
                norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
                norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize

                if color_channel_stft is None:
                    color_channel_stft = norm_stft
                else:
                    color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)
                    
            full_norm_stft[start_point_number, : , :, color_channel] = color_channel_stft
            color_channel_stft = None
            
        start_point_number = start_point_number + 1
        
    return full_norm_stft, np.array(subject_file["labels"][trial])


dsize = (512, 128)

data, labels = trial_file_processor(raw_files_addr+"s01.dat", 5)
print(data.shape)
data1 = cv2.resize(data[0], dsize)
data2 = cv2.resize(data[1], dsize)
for file_name in file_names:
    subject_name = file_name.split(".")[0]
    for trial in range(0, 40):
        data, labels = trial_file_processor(raw_files_addr+file_name, trial)
        for sample in range(0, data.shape[0]):
            data_label_dict = {"data":cv2.resize(data[sample], dsize), "labels":labels}
            with open(transformed_files_addr+subject_name+"t"+str(trial)+"s"+str(sample), 'wb') as handle:
                pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
################# Only EEG Data #################

def trial_file_processor(file_addr, trial):
    
    offset = 3
    window_length = 10
    start_points = [15+offset, 25+offset, 35+offset]
    
    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = 0 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(129, 352, 3))
    
    
    for color_channel in range(0, 2):
        for channel in range(0, 32):
            stft = librosa.stft(subject_file["data"][trial, channel,
                                          128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                          n_fft=256, hop_length=125)
            stft_db = librosa.amplitude_to_db(abs(stft))
            norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
            norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize

            if color_channel_stft is None:
                color_channel_stft = norm_stft
            else:
                color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)

        full_norm_stft[: , :, color_channel] = color_channel_stft
        color_channel_stft = None

    return full_norm_stft, subject_file["labels"][trial] 


raw_files_addr = "Address/to/preprocessed_DEAP_data"
transformed_files_addr = "E:/Hamavar/DEAP_Dataset/onlyEEG_transformed_data/"
file_names = os.listdir(raw_files_addr)

for file_name in file_names:
    for trial in range(0, 40):
        data, labels = trial_file_processor("Address/to/preprocessed_DEAP_data"+file_name, trial)
        data_label_dict = {"data":data, "labels":labels}
        subject_name = file_name.split(".")[0]
        with open(transformed_files_addr+subject_name+"t"+str(trial), 'wb') as handle:
            pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
############### Preprocessing Data ################

def trial_file_processor(file_addr, trial):
    
    offset = 3
    window_length = 10
    start_points = [15+offset, 25+offset, 35+offset]
    
    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = -1 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(129, 440, 3))
    
    
    for color_channel in range(0, 2):
        for channel in range(0, 40):
            stft = librosa.stft(subject_file["data"][trial, channel,
                                          128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                          n_fft=256, hop_length=125)
            stft_db = librosa.amplitude_to_db(abs(stft))
            norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
            norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize

            if color_channel_stft is None:
                color_channel_stft = norm_stft
            else:
                color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)

        full_norm_stft[: , :, color_channel] = color_channel_stft
        color_channel_stft = None

    return full_norm_stft, subject_file["labels"][trial]


raw_files_addr = "Address/to/preprocessed_DEAP_data"
transformed_files_addr = "Address/to/save/data"
file_names = os.listdir(raw_files_addr)

for file_name in file_names:
    for trial in range(0, 40):
        data, labels = trial_file_processor("Address/to/preprocessed_DEAP_data"+file_name, trial)
        data_label_dict = {"data":data, "labels":labels}
        subject_name = file_name.split(".")[0]
        with open(transformed_files_addr+subject_name+"t"+str(trial), 'wb') as handle:
            pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
############# sample data loading #############
transformed_files_addr = "Address/to/save/data"
with open(transformed_files_addr+"s01t1s4", 'rb') as handle:
    test = pickle.load(handle)

print(test["labels"].shape)

In [ ]:
############## data generator ################
transformed_files_addr = "Address/to/save/data"

def train_test_spliter(addr, train_ratio, val_ratio):
    
    files = os.listdir(addr)
    number_of_files = len(files)
    random.shuffle(files)
    end_of_train_files = math.floor(train_ratio*number_of_files)
    train_files = files[0:end_of_train_files]
    validation_files = files[end_of_train_files:end_of_train_files+math.floor(val_ratio*number_of_files)]
    test_files = files[end_of_train_files+math.floor(val_ratio*number_of_files):]
    
    return train_files, validation_files, test_files
    
def OLO_train_test_spliter(addr, val_one, test_one):
    # val_one and test_one should be between 1 and 32
    files = os.listdir(addr)
    
    
    train_files = files
    
    val_files = list()
    val_re_string = None
    
    test_files = list()
    test_re_string = None
    
    if val_one < 10:
        val_re_string = f"s0{val_one}.+"
    else:
        val_re_string = f"s{val_one}.+"
        
    if test_one < 10:
        test_re_string = f"s0{test_one}.+"
    else:
        test_re_string = f"s{test_one}.+"
    
    for file in files:
        if re.search(val_re_string, file):
            val_files.append(file)
        
        if re.search(test_re_string, file):
            test_files.append(file)
            
    
    for index in range(0, len(val_files)):
            train_files.remove(val_files[index])
            train_files.remove(test_files[index])
    
    return train_files, val_files, test_files
    
    

    
class ERDataGenerator(keras.utils.Sequence):
    
    def __init__(self, files_addrs, feature_files_addr, batch_size=32):
        self.files_addrs = files_addrs
        self.feature_files_addr = feature_files_addr
        self.batch_size = batch_size
        
        
    def number_of_samples(self):
        return len(self.files_addrs)


    def __len__(self):
        return int(np.ceil(len(self.files_addrs) / self.batch_size))

    def __getitem__(self, index):
        batch_files_addrs = self.files_addrs[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_data, batch_labels = self.__data_generation(batch_files_addrs)

        return batch_data, batch_labels

    

    def __data_generation(self, batch_file_addrs):
        batch_data = list()
        batch_labels = list()

        for batch_file_addr in batch_file_addrs:
            with open(self.feature_files_addr+batch_file_addr, 'rb') as handle:
                loaded_data = pickle.load(handle)
                batch_data.append(loaded_data["data"])
                batch_labels.append(loaded_data["labels"])

        return (np.array(batch_data), np.array(batch_labels)) 
    
    
    
batch_size = 32    
train_files, validation_files, test_files = OLO_train_test_spliter(transformed_files_addr,1, 2)


test_samples = len(test_files)
train_gen = ERDataGenerator(train_files, transformed_files_addr)
validation_gen = ERDataGenerator(validation_files, transformed_files_addr)
test_gen = ERDataGenerator(test_files, transformed_files_addr)


In [ ]:
class ER_OCC_DataGenerator(keras.utils.Sequence):
    
    def __init__(self, files_addrs, feature_files_addr, o_channels_freqs_pairs, batch_size=32):
        
        self._channel_length = 11
        self._freq_bands = [[0, 129]]
        
        self.o_channels_freqs_pairs = o_channels_freqs_pairs
        
        self.files_addrs = files_addrs
        self.feature_files_addr = feature_files_addr
        self.batch_size = batch_size
        
    
    def number_of_samples(self):
        return len(self.files_addrs)


    def __len__(self):
        return int(np.ceil(len(self.files_addrs) / self.batch_size))

    def __getitem__(self, index):
        batch_files_addrs = self.files_addrs[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_data, batch_labels = self.__data_generation(batch_files_addrs)

        return batch_data, batch_labels

    
    def __data_generation(self, batch_file_addrs):
        batch_data = list()
        batch_labels = list()
    
        for batch_file_addr in batch_file_addrs:
            with open(self.feature_files_addr+batch_file_addr, 'rb') as handle:
                loaded_data = pickle.load(handle)
                batch_data.append(loaded_data["data"])
                batch_labels.append(loaded_data["labels"])
        
        occ_data = self._occ(np.array(batch_data), self.o_channels_freqs_pairs)
        return (occ_data, np.array(batch_labels)) 
    
    def _occ(self, data, o_channels_freq_pairs):
        masked_data = np.zeros(shape=(data.shape[0], data.shape[1], data.shape[2], data.shape[3]))
        occ_mask_1D = np.ones(shape=(data.shape[1], data.shape[2]))
        occ_mask = np.zeros(shape=(data.shape[1], data.shape[2], data.shape[3]))
        for channel_freq_pair in o_channels_freq_pairs:
            channel = channel_freq_pair[0]
            freq_band = channel_freq_pair[1]
            low_freq = self._freq_bands[freq_band][0]
            high_freq = self._freq_bands[freq_band][1]
            occ_mask_1D[low_freq:high_freq, channel*self._channel_length:(channel+1)*self._channel_length] = 0
        
        for color_channel in range(0, data.shape[3]):
            occ_mask[:, :, color_channel] = occ_mask_1D
        
        for sample in range(0, data.shape[0]):
            masked_data[sample, :, :, :] = data[sample, :, :, :]*occ_mask 
            
        return masked_data
        
        

occ_gen = ER_OCC_DataGenerator(test_files, transformed_files_addr, [[2, 0], [5, 0], [8, 0], [12, 0], [14, 0], [16, 0],
                                                                    [18, 0], [19, 0], [23, 0], [25, 0], [28, 0], [32, 0], [33, 0], [35, 0],
                                                                    [36, 0]], batch_size=32)

In [ ]:
input_layer = keras.Input(shape=(129, 440, 3))

x = tf.keras.layers.Conv2D(16, kernel_size=(5, 11), padding="same", activation="relu")(input_layer)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), padding="same",activation="relu")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), padding="same", activation="relu")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Dense(units=512, activation="relu")(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)
x = tf.keras.layers.Dense(units=256, activation="relu")(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)
x = tf.keras.layers.Dense(units=128, activation="relu")(x)

x = tf.keras.layers.Dense(units=4)(x)

Model = tf.keras.models.Model(input_layer, x)
Model.compile(loss='mae', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005)) # tf.keras.optimizers.Adam(learning_rate=0.007)

Model.summary()

In [ ]:
train_hist = Model.fit(train_gen, validation_data=validation_gen, epochs=600)

In [ ]:
number_of_epochs = 200
train_mae = np.zeros(shape=(number_of_epochs, 1))
val_mae = np.zeros(shape=(number_of_epochs, 1))

for epoch in range(0, number_of_epochs):
    print(f"epoch number:{epoch+1}")
    K.clear_session()
    train_hist = Model.fit(train_gen, steps_per_epoch=steps_per_epoch)
    train_mae[epoch, 0] = train_hist.history['loss'][0] 
    val_data, val_labels = next(validation_gen)
    test_hist = Model.evaluate(val_data, val_labels)
    val_mae[epoch, 0] = test_hist 

In [ ]:
plt.plot(train_hist.history['loss'])
plt.plot(train_hist.history['val_loss'])

In [ ]:
predicted_values = np.zeros(shape=(test_samples, 4))
test_labels = np.zeros(shape=(test_samples, 4))

counter = 0
for data, labels in test_gen:
    predicted = Model.predict(data)
    
    predicted_values[counter*batch_size:(counter+1)*batch_size, :] = predicted
    test_labels[counter*batch_size:(counter+1)*batch_size, :] = labels
    counter = counter + 1

In [ ]:
predicted_values = np.zeros(shape=(test_samples, 4))
test_labels = np.zeros(shape=(test_samples, 4))

counter = 0
for data, labels in occ_gen:
    predicted = Model.predict(data)
    
    predicted_values[counter*batch_size:(counter+1)*batch_size, :] = predicted
    test_labels[counter*batch_size:(counter+1)*batch_size, :] = labels
    counter = counter + 1

In [ ]:
def accuracy_evaluation_percentage(true_labels, predicted_labels):
    labels_dimension = 4
    false_predictions = np.zeros(shape=(1, labels_dimension))
    total_numbers_of_test_samples = len(predicted_labels)
    
    for prd_idx in range(0, total_numbers_of_test_samples):
        for label_idx in range(0, labels_dimension):
            if ((predicted_labels[prd_idx, label_idx]>5) and (true_labels[prd_idx, label_idx]<5)) or \
            ((predicted_labels[prd_idx, label_idx]<5) and (true_labels[prd_idx, label_idx]>5)):
                false_predictions[0, label_idx] = false_predictions[0, label_idx] + 1

    return ((total_numbers_of_test_samples - false_predictions)/total_numbers_of_test_samples)*100



def predict_labels(Model, generator):
    predicted_labels = np.zeros(shape=(generator.number_of_samples(), 4))
    true_labels = np.zeros(shape=(generator.number_of_samples(), 4))

    counter = 0
    for data, labels in generator:
        predicted = Model.predict(data)
    
        predicted_labels[counter*batch_size:(counter+1)*batch_size, :] = predicted
        true_labels[counter*batch_size:(counter+1)*batch_size, :] = labels
        counter = counter + 1
        
    return true_labels, predicted_labels
    

In [ ]:
# calculating the normal loss values
number_of_channels = 40


true_labels, predicted_labels = predict_labels(Model, test_gen)  
ground_truth = accuracy_evaluation_percentage(true_labels, predicted_labels)

channel_freq_occ_loss = np.zeros(shape=(9, number_of_channels, 4))
for channel_index in range(0, 40):
    for freq_index in range(0, 1):
        occ_gen = ER_OCC_DataGenerator(test_files, transformed_files_addr, [[channel_index, freq_index]], batch_size=32)
        
        true_labels, predicted_labels = predict_labels(Model, occ_gen)
        channel_freq_occ_loss[freq_index, channel_index, :] = ground_truth - accuracy_evaluation_percentage(true_labels, predicted_labels)
    

reshape_loss_values = channel_freq_occ_loss.reshape((channel_freq_occ_loss.shape[0]*channel_freq_occ_loss.shape[1], channel_freq_occ_loss.shape[2])) 
    
for feeling_index in range(0, 3):
    reshape_loss_values[:, feeling_index] = (reshape_loss_values[:, feeling_index] - reshape_loss_values[:, feeling_index].min())/(reshape_loss_values[:, feeling_index].max() - reshape_loss_values[:, feeling_index].min())
    
norm_loss_values = reshape_loss_values.reshape((channel_freq_occ_loss.shape[0], channel_freq_occ_loss.shape[1], channel_freq_occ_loss.shape[2]))
norm_loss_values = norm_loss_values*255
print(norm_loss_values.shape)

In [ ]:
Model.evaluate(test_gen)

In [ ]:
channel_length = 11
freq_height = 129
freq_bands = [[0, 129]]#[[0, 4], [4, 8], [8, 14], [14, 30], [30, 50], [50, 70], [70, 90], [90, 110], [110, 129]]

channel_heat_map = np.ones(shape=(freq_height, channel_length))
heat_map = np.zeros(shape=(freq_height, 40*channel_length, 3))
for frequency_index in range(0, len(freq_bands)):
    for channel_index in range(0, 40):
        for color_index in range(0, 3):
            heat_map[freq_bands[frequency_index][0]:freq_bands[frequency_index][1], channel_index*channel_length:(channel_index+1)*channel_length, color_index] = \
            channel_heat_map[freq_bands[frequency_index][0]:freq_bands[frequency_index][1],:]*norm_loss_values[frequency_index, channel_index, color_index]
       

In [ ]:
accuracy_evaluation_percentage(true_labels, predicted_labels)

In [ ]:
############### SC-Preprocessing Data ################

def trial_file_processor(file_addr, trial):
    
    offset = 3
    window_length = 10
    start_points = [15+offset, 25+offset, 35+offset]
    selected_channels = np.array([ 1, 2, 4, 7, 8, 11, 12, 13, 15, 16, 17, 18, 19, 22, 27, 28, 30, 31, 32, 33, 37, 38])
 
    
    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = -1 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(129, 242, 3))
    
    
    for color_channel in range(0, 2):
        for channel in selected_channels:
            stft = librosa.stft(subject_file["data"][trial, channel,
                                          128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                          n_fft=256, hop_length=125)
            stft_db = librosa.amplitude_to_db(abs(stft))
            norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
            norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize

            if color_channel_stft is None:
                color_channel_stft = norm_stft
            else:
                color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)

        full_norm_stft[: , :, color_channel] = color_channel_stft
        color_channel_stft = None

    return full_norm_stft, subject_file["labels"][trial]


raw_files_addr = "Address/to/preprocessed_DEAP_data"
transformed_files_addr = "E:/Hamavar/DEAP_Dataset/selectedChannelsData/median/"
file_names = os.listdir(raw_files_addr)

for file_name in file_names:
    for trial in range(0, 40):
        data, labels = trial_file_processor("Address/to/preprocessed_DEAP_data"+file_name, trial)
        data_label_dict = {"data":data, "labels":labels}
        subject_name = file_name.split(".")[0]
        with open(transformed_files_addr+subject_name+"t"+str(trial), 'wb') as handle:
            pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

